# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
import gradio as gr

In [ ]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from pathlib import Path

In [ ]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [ ]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

In [ ]:
# set up environment
load_dotenv(override=True)
openai = OpenAI()
api_key = os.getenv('OPENAI_API_KEY')
datafolder = Path('C:\\Projects\\llm_engg\\llm_engineering\\Pdata')
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# Initialize conversation history
conversation_history = [{"role": "system", "content": "You are a helpful assistant. Respond in markdown."}]

# here is the question; type over this to ask something new
system_prompt = "You are an assistant that analyzes the code. Respond in markdown.\
Include details if you have the information."

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""
def getFileContent(fp):
    
    with open(fp, 'r') as file:
        file_content = file.read()
        return file_content
    return ""
    
# Get gpt-4o-mini to answer, with streaming
def chatGptResponse(qt,file):
    
    conversation_history.append({"role": "user", "content": qt})
    if file: conversation_history.append({"role": "user", "content": getFileContent(file)})
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=conversation_history,
        stream=True,
    )
    response = ""
    
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        
        yield response
    conversation_history.append({"role": "assistant", "content": response})

In [ ]:
view = gr.Interface(
    fn=chatGptResponse,
    inputs=[gr.Textbox(label="Your message:"),gr.File(label="Upload file")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()
#view.launch(server_name="10.75.32.62", server_port=7866)

In [ ]:
conversation_history